In [ ]:
import os
%load_ext autoreload
%autoreload 2

In [ ]:
# make deterministic
from mingpt.utils import set_seed
set_seed(44)

In [ ]:
import math
import time
import numpy as np
from copy import deepcopy
import pickle
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
from torch.nn import functional as F
from torch.utils.data import Subset
from tqdm import tqdm
from matplotlib import pyplot as plt

from data import get_othello, plot_probs, plot_mentals
from data.othello import permit, start_hands, OthelloBoardState, permit_reverse
from mingpt.dataset import CharDataset
from mingpt.model import GPT, GPTConfig, GPTforProbeIA
from mingpt.utils import sample, intervene, print_board
from mingpt.probe_model import BatteryProbeClassification, BatteryProbeClassificationTwoLayer

plt.rc('text', usetex=True)
plt.rc('font', **{'size': 14.0})
plt.rc('text.latex', preamble=r'\usepackage{lmodern}')

In [ ]:
championship = False

In [ ]:
mid_dim = 128
how_many_history_step_to_use = 99
exp = f"state_tl{mid_dim}"
if championship:
    exp += "_championship"

## Load a game from intervention benchmark and select an intervention configuration

In [ ]:
root_folder = "./ckpts/"

In [ ]:
with open("intervention_benchmark.pkl", "rb") as input_file:
    dataset = pickle.load(input_file)

In [ ]:
case_id = 777
wtd = {
    "intervention_position": permit_reverse(dataset[case_id]["pos_int"]), 
    "intervention_from": dataset[case_id]["ori_color"], 
    "intervention_to": 2 - dataset[case_id]["ori_color"], 
}
completion = dataset[case_id]["history"]
print(wtd)

## Load nonlinear probes

In [ ]:
probes = {}
layer_s = 4
layer_e = 9
for layer in range(layer_s, layer_e):
    p = BatteryProbeClassificationTwoLayer(torch.cuda.current_device(), probe_class=3, num_task=128, mid_dim=mid_dim)
    load_res = p.load_state_dict(torch.load(f"{root_folder}/battery_othello/{exp}/layer{layer}/checkpoint.ckpt"))
    p.eval()
    probes[layer] = p

## Load trained models

In [ ]:
# othello = get_othello(ood_perc=.2, data_root="data/othello_pgn", wthor=False)
# othello = get_othello(ood_perc=0., data_root=None, wthor=False, ood_num=1)
othello = get_othello(data_root="data/othello_championship", nfiles=3)
train_dataset = CharDataset(othello)

mconf = GPTConfig(121, 59, n_layer=8, n_head=8, n_embd=512)

models = {}
for layer in range(layer_s, layer_e):
    model = GPTforProbeIA(mconf, probe_layer=layer)
    # model = GPT(mconf)
    load_res = model.load_state_dict(torch.load(f"{root_folder}/gpt_synthetic.ckpt" if not championship else "./ckpts/gpt_championship.ckpt"))
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        model = model.to(device)
    _ = model.eval()
    models[layer] = model

In [ ]:
def to_board_1(seq):
    return [x % 64 for x in seq]

evaluation_dataset = [to_board_1(seq[:38]) for seq in train_dataset.data[:1000]]

### Check the partial game progression

In [ ]:
ab = OthelloBoardState()
completion = evaluation_dataset[1]
ab.update(completion, prt=True)

In [ ]:
partial_game = torch.tensor([train_dataset.stoi[s] for s in completion], dtype=torch.long).to(device)

In [ ]:
np.array(completion)

In [ ]:
partial_game

In [ ]:
whole_mid_act = models[layer_s].forward_1st_stage(partial_game[None, :])  # [B, T, F=512]
mid_act = whole_mid_act[0, -1]
pre_intv_logits = probes[5](mid_act[None, :])[0].squeeze(0)  # [64, 3]
probed_board = pre_intv_logits.argmax(axis=1)
probed_states = (probed_board - 1).cpu().numpy()
board = OthelloBoardState()
board.state = probed_states[:64].reshape([8,8])
board.__print__()
board.state = probed_states[64:].reshape([8,8])
board.__print__()

In [ ]:
partial_game

In [ ]:
def probing_board_score(game):
    true_board = OthelloBoardState()
    true_board.update(game)
    
    def score_for_indexes(true_board, game_indexes):
        whole_mid_act = models[layer_s].forward_1st_stage(game_indexes[None, :])  # [B, T, F=512]
        mid_act = whole_mid_act[0, -1]
        pre_intv_logits = probes[5](mid_act[None, :])[0].squeeze(0)
        probed_board = pre_intv_logits.argmax(axis=1)
        probed_states = (probed_board - 1).cpu().numpy()
    #     return true_board.state, probed_states[64:].reshape([8,8])
        return (true_board.state.reshape(-1) == probed_states[64:]).sum() / 64
    
    game_indexes1 = torch.tensor([train_dataset.stoi[s] for s in game], dtype=torch.long).to(device)
    game_indexes2 = torch.tensor([train_dataset.stoi[s+64] for s in game], dtype=torch.long).to(device)
    return score_for_indexes(true_board, game_indexes2),score_for_indexes(true_board, game_indexes1)

def mutual_representation_score(dataset):
    acc, baseline = zip(
        *[probing_board_score(x) for x in dataset]
    )
    return np.array(acc).mean(), np.array(baseline).mean()

mutual_representation_score(evaluation_dataset)

In [ ]:
def rotate_matrix_90_deg(matrix):
    # Step 1: Transpose the matrix
    transposed_matrix = [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]
    
    # Step 2: Reverse each row
    rotated_matrix = [row[::-1] for row in transposed_matrix]
    
    return np.array(rotated_matrix)

In [ ]:
def probing_board_score(game):
    true_board = OthelloBoardState()
    true_board.update(game)
    
    def score_for_indexes(true_board, game_indexes):
        whole_mid_act = models[layer_s].forward_1st_stage(game_indexes[None, :])  # [B, T, F=512]
        mid_act = whole_mid_act[0, -1]
        pre_intv_logits = probes[5](mid_act[None, :])[0].squeeze(0)
        probed_board = pre_intv_logits.argmax(axis=1)
        probed_states = (probed_board - 1).cpu().numpy()
        true_board_rotated_90deg = rotate_matrix_90_deg(true_board.state)
        true_board_rotated_180deg = rotate_matrix_90_deg(true_board_rotated_90deg)
        true_board_rotated_270deg = rotate_matrix_90_deg(true_board_rotated_180deg)
        return (
            (true_board.state.reshape(-1) == probed_states[64:]).sum() / 64,
            (true_board_rotated_90deg.reshape(-1) == probed_states[64:]).sum() / 64,
            (true_board_rotated_180deg.reshape(-1) == probed_states[64:]).sum() / 64,
            (true_board_rotated_270deg.reshape(-1) == probed_states[64:]).sum() / 64,
        )
    
    game_indexes1 = torch.tensor([train_dataset.stoi[s] for s in game], dtype=torch.long).to(device)
    game_indexes2 = torch.tensor([train_dataset.stoi[s+64] for s in game], dtype=torch.long).to(device)
    return score_for_indexes(true_board, game_indexes2)[0], *score_for_indexes(true_board, game_indexes1)

def mutual_representation_score(dataset):
    scores = zip(
        *[probing_board_score(x) for x in dataset]
    )
    return [np.array(s).mean() for s in scores]

mutual_representation_score(evaluation_dataset)

### Check pre-intervention ground-truth legal next-steps and the predicted ones

In [ ]:
pre_intv_valids = [permit_reverse(_) for _ in ab.get_valid_moves()]
pre_intv_valids

In [ ]:
partial_game

In [ ]:
pre_intv_pred, _ = model(partial_game[None, :])  # [B, T, F=512]
# no history activations used here, that's why the prediction map is different to across layers
pre_intv_pred = pre_intv_pred[0, -1, 1:]
padding = torch.zeros(2).cuda()
# pre_intv_pred = torch.softmax(pre_intv_pred, dim=0)  # keep logits
pre_intv_pred = torch.cat(
    [
        pre_intv_pred[:27],
        padding,
        pre_intv_pred[27:33],
        padding,
        pre_intv_pred[33:27+60],
        padding,
        pre_intv_pred[27+60:33+60],
        padding,
        pre_intv_pred[33+60:]
    ], dim=0)

In [ ]:
plt.style.use(['classic'])

In [ ]:
fig=plt.figure(figsize=(10, 6), dpi= 80, facecolor='w', edgecolor='k')
vv = 10
gems = {"pre": pre_intv_pred.detach().cpu().numpy()[:64].reshape(8, 8)}
sns.heatmap(pre_intv_pred.detach().cpu().numpy()[:64].reshape(8, 8), vmin=-vv, vmax=vv, 
            yticklabels=list("ABCDEFGH"), xticklabels=list(range(1,9)), square=True, 
            annot=True, fmt=".2f")

In [ ]:
fig=plt.figure(figsize=(10, 6), dpi= 80, facecolor='w', edgecolor='k')
vv = 10
gems = {"pre": pre_intv_pred.detach().cpu().numpy()[64:].reshape(8, 8)}
sns.heatmap(pre_intv_pred.detach().cpu().numpy()[64:].reshape(8, 8), vmin=-vv, vmax=vv, 
            yticklabels=list("ABCDEFGH"), xticklabels=list(range(1,9)), square=True, 
            annot=True, fmt=".2f")

## Intervene on all board positions and save prediction logits
0 for white; 1 for blank; 2 for black

In [ ]:
htd = {"lr": 1e-3, "steps": 1000, "reg_strg": 0.2}

In [ ]:
board_probing = []
for square in [permit_reverse(_) for _ in range(64)]:
    p = probes[layer_s]
    whole_mid_act = models[layer_s].forward_1st_stage(partial_game[None, :])  # [B, T, F=512]
    mid_act = whole_mid_act[0, -1]
    pre_intv_logits = p(mid_act[None, :])[0].squeeze(0)  # [64, 3]
    break
pre_intv_logits.argmax(axis=1)

In [ ]:
partial_game

In [ ]:
whole_mid_act = models[layer_s].forward_1st_stage(partial_game[None, :])  # [B, T, F=512]
mid_act = whole_mid_act[0, -1]
pre_intv_logits = probes[6](mid_act[None, :])[0].squeeze(0)  # [64, 3]
probed_board = pre_intv_logits.argmax(axis=1)

In [ ]:
probed_board.reshape([8,8])

In [ ]:
for square in [permit_reverse(_) for _ in range(64)]:
    wtd["intervention_position"] = square
    move = permit(wtd["intervention_position"])
    r, c = move // 8, move % 8
    wtd["intervention_from"] = ab.state[r, c] + 1
    wtd["intervention_to"] = 2 - wtd["intervention_from"]
    wtd_list = [wtd]
    
    p = probes[layer_s]
    whole_mid_act = models[layer_s].forward_1st_stage(partial_game[None, :])  # [B, T, F=512]

    # intervene at the earlest interested layer 
    mid_act = whole_mid_act[0, -1]
    pre_intv_logits = p(mid_act[None, :])[0].squeeze(0)  # [64, 3]
    labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)
    new_mid_act = mid_act.clone()
    for wtd in wtd_list:
        new_mid_act = intervene(p, new_mid_act, labels_pre_intv, wtd, htd, plot=True)
        pre_intv_logits = p(new_mid_act[None, :])[0].squeeze(0)  # [64, 3]
        labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)

    # swap in 
    whole_mid_act[0, -1] = new_mid_act

    for i, layer in enumerate(range(layer_s, layer_e - 1)):  # 4, 5, 6, 7, indices of the layers to be passed
        p = probes[layer+1]
        whole_mid_act = models[layer_s].forward_2nd_stage(whole_mid_act, layer, layer+1)[0]  # [1, T, F=512]

        # intervene the output of the features freshly out
        mid_act = whole_mid_act[0, -1]
        pre_intv_logits = p(mid_act[None, :])[0].squeeze(0)  # [64, 3]
        labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)
        new_mid_act = mid_act.clone()
        for wtd in wtd_list:
            new_mid_act = intervene(p, new_mid_act, labels_pre_intv, wtd, htd, plot=True)
            pre_intv_logits = p(new_mid_act[None, :])[0].squeeze(0)  # [64, 3]
            labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)
        # swap in 
        whole_mid_act[0, -1] = new_mid_act

    tb_resumed = whole_mid_act
    post_intv_pred, _ = models[layer_s].predict(tb_resumed)
    post_intv_pred = post_intv_pred[0, -1, 1:]
    # post_intv_pred = torch.softmax(post_intv_pred, dim=0)
    post_intv_pred = torch.cat([post_intv_pred[:27], padding, post_intv_pred[27:33], padding, post_intv_pred[33:]], dim=0)
    if 0:
        gems[square] = torch.softmax(post_intv_pred, dim=0).detach().cpu().numpy().reshape(8, 8)
    else:   
        gems[square] = post_intv_pred.detach().cpu().numpy().reshape(8, 8)

## Plot Attribution via Intervention heatmaps for all legal next-moves

In [ ]:
tbu = pre_intv_valids
total = len(tbu)
rows = math.ceil(total / 4)

fig, axs = plt.subplots(rows, 4, figsize=(30, rows * 8), dpi= 80, facecolor='w', edgecolor='k')
axs = axs.flat

for i, tobe_tcaved in enumerate(tbu):
    pred = permit(tobe_tcaved)  # 0-63
    r_pred, c_pred = pred // 8, pred % 8

    tbp = np.zeros((8, 8), )
    pre = gems["pre"]
    for k, w in gems.items():
        if k == "pre":
            continue
        move = permit(k)  # 0-63
        r, c = move // 8, move % 8
        tbp[r, c] = - w[r_pred, c_pred] + pre[r_pred, c_pred]
    ab.plot_hm(axs[i], tbp.flatten(), permit(tobe_tcaved), logit=True)